1. The data. I already prepared it during the tutorial.

In [ ]:
# link to GitHub
# No need to download this dat YET
link="https://github.com/DACSS-CSSmeths/Spatial-Exploring/raw/refs/heads/main/maps/countriesCIA.gpkg"

# we use geopandas
# geopandas is in Colab, you need to install it in Anaconda.
import geopandas as gpd

# read the data
countriesCIA=gpd.read_file(link, layer='cia')

Take a look at it:

In [ ]:
# see some rows
countriesCIA.head()

In [ ]:
# the data types
countriesCIA.info()

2. The variable for the homework

In [ ]:
# we want to CUT this variable into bins
countriesCIA.obesityAdults_rate.describe()

For the homework you may need to install pysal in Colab and in Anaconda.

In [ ]:
# !pip install pysal # write a "#" (no quotations) after the installation

I requested to decide what is the best scheme if I wanted to split the data into 5 intervals.

Below you see



```
theVar=countriesCIA.TobaccoUse_perc
```

Change the variable



In [ ]:

import mapclassify
import numpy as np

np.random.seed(12345) # so we all get the same results!

# let's try 5 bins
K=5

theVar=countriesCIA.TobaccoUse_perc


# same interval width, easy interpretation
ei5 = mapclassify.EqualInterval(theVar, k=K)
# same interval width based on standard deviation, easy - but not as the previous one, poor when high skewness
msd = mapclassify.StdMean(theVar)
# interval width varies, counts per interval are close, not easy to grasp, repeated values complicate cuts
q5=mapclassify.Quantiles(theVar,k=K)
# based on similarity, good for multimodal data
mb5 = mapclassify.MaximumBreaks(theVar, k=K)
# based on similarity, optimizer
fj5 = mapclassify.FisherJenks(theVar, k=K)
# based on similarity, optimizer
jc5 = mapclassify.JenksCaspall(theVar, k=K)
# based on similarity, optimizer
mp5 = mapclassify.MaxP(theVar, k=K)

###### based on similarity, good for skewed data
ht = mapclassify.HeadTailBreaks(theVar) # no K needed

With these results, you choose the best (select the shortest bar).

**WHICH ONE IS THE BEST?**

In [ ]:
import pandas as pd

# Create a list of classifiers
class5 = ei5,msd, q5,mb5,  ht, fj5, jc5, mp5

# Collect ADCM for each classifier
fits = np.array([ c.adcm for c in class5])
# Convert ADCM scores to a DataFrame
adcms = pd.DataFrame(fits)
# Add classifier names
adcms['classifier'] = [c.name for c in class5]
# Add column names to the ADCM
adcms.columns = ['ADCM', 'Classifier']

# see the results
adcms.sort_values('ADCM').plot.barh(x='Classifier')

The winner is one of these



```
ei5,msd, q5,mb5,  ht, fj5, jc5, mp5
```



I wrote:

```
countriesCIA['obesity_code']=fj5.yb
```
when FisherJenks was the best....what would yo change below if we are using 'countriesCIA.TobaccoUse_perc'?


In [ ]:
# map of labels for the levels
newLevels={0:'1.very low',1:'2.low', 2:'3.average',3:'4.high', 4:'5.very high'}

# get the values
countriesCIA['tobacco_code']=fj5.yb

# a copy in another column
countriesCIA=countriesCIA.assign(obesity_levels=countriesCIA.tobacco_code)

# recoding
countriesCIA.replace({'tobacco_levels':newLevels}, inplace=True)


If the winner was FisherJenks you go ahead and answer the question:
**Find the countries that are doing the best in all three variables?** (use the columns ending in _code_ for simplicity)